<a href="https://colab.research.google.com/github/MartinaCecchetto/DMDV/blob/main/Code/MongoDB_Insert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Librerie utilizzate
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
import pandas as pd
import re
import datetime
from dateutil.parser import parse
import pymongo
from pymongo import MongoClient

In [ ]:
#Driver utilizzato per lo scraping della pagina 'http://cycling.data.tfl.gov.uk/'
chromedriver = ('/Users/martinacecchetto/Desktop/Università/Data_Management/Progetto/chromedriver')

In [ ]:
#Scraping con Selenium e successivamente Beautiful Soup per cercare i file a noi utili
url='http://cycling.data.tfl.gov.uk/'
driver = webdriver.Chrome(executable_path=chromedriver)
driver.get(url)
time.sleep(5)
innerHTML = driver.execute_script("return document.body.innerHTML")
driver.quit()
soup = BeautifulSoup(innerHTML, "html.parser")
trs=soup.find_all('tr',{'data-level':'3'})
d=[]
t=[]
dim=[]
for tr in trs:
    if tr.find('a', href=re.compile('usage-stats' and 'Extract')):
        link=tr.find('a', href=re.compile('usage-stats' and 'Extract'))
        d.append(link.get('href'))
        t.append(parse(tr.find_all('td')[1].text))
        dim.append(float(tr.find_all('td')[2].text.split()[0]))
d=[]
t=[]
dim=[]
for tr in trs:
    if tr.find('a', href=re.compile('usage-stats' and 'Extract')):
        link=tr.find('a', href=re.compile('usage-stats' and 'Extract'))
        d.append(link.get('href'))
        t.append(parse(tr.find_all('td')[1].text))
        dim.append(float(tr.find_all('td')[2].text.split()[0]))
files=pd.DataFrame({'file':d,'data_modified':t, 'dim_MB':dim})
files=files.sort_values('data_modified').reset_index(drop=True)


In [ ]:
#Elenco dei file disponibili da Gennaio 2018 al 28 Luglio 2020
files.iloc[118:-1]

In [ ]:
#Unione dei file in una unica lista
li=[]
for row in files.iloc[118:-1].iterrows():
    print(row[1][0])
    df = pd.read_csv(row[1][0],sep=',',index_col=None, header=0)
    li.append(df)
    

In [ ]:
#Trasfomazione della lista in un dataframe
frame = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
#Creo dizionario journeys da caricare du mongodb
journeys=frame.to_dict(orient="records")

In [ ]:
#Mi collengo al database distribuito "bikesharingDB"
client = MongoClient('localhost', 27027)
db = client.bikesharingDB

In [ ]:
#Caricamento dei dati nella collezione "shardedCollection" del database
db.shardedCollection.insert_many(journeys)